---
title: "Skills"
format: html
editor: visual
---




Through my coursework at UBC, I am proficient in using R, Python, and GIS softwares to tackle data processing and geospatial analysis. Here are a few excerpts of my work:

#Python

My research into forest disturbance ecology relied heavily on the Vegetation Resource Index produced by the Government of Canada. These data are available as geodatabases which is best handled using python to extract variables of interest. 


In [ ]:
# Python script for filtering vegetation data and spatial analysis
import os
import arcpy
py -m pip install jupyter
# Set the directory path where the data is stored
data_dir_path = r"path/to/working/directory"
print(f"Data Directory: {data_dir_path}")

# Set ArcPy workspace to a specific geodatabase
arcpy.env.workspace = os.path.join(data_dir_path, "veg_comp_2010.gdb")

# Allow overwriting of outputs
arcpy.env.overwriteOutput = True

# Define a scratch workspace for temporary files
scratch_gdb_fpath = os.path.join(data_dir_path, "scratch.gdb")
print(f"Scratch Geodatabase Path: {scratch_gdb_fpath}")

# List all feature classes in the current workspace
fc_list = arcpy.ListFeatureClasses()
print(f"Feature Classes in Workspace: {fc_list}")

# Check out the Spatial Analyst extension (if required)
arcpy.CheckOutExtension("Spatial")

# Define the input shapefile and output shapefile paths
input_shapefile = os.path.join(data_dir_path, "VEG_COMP_LYR_R1_POLY.shp")
output_shapefile = os.path.join(data_dir_path, "AspenDom_2010_VRI.shp")

# Specify the attribute field and value for filtering
attribute_name = "FHF"  
attribute_value = "ID6"

# Create a SQL query to filter the features
query = f"{attribute_name} = '{attribute_value}'"

# Perform the selection and save it to a new shapefile
arcpy.Select_analysis(in_features=input_shapefile, out_feature_class=output_shapefile, where_clause=query)

print(f"Filtered polygons with {attribute_name} = {attribute_value} saved to {output_shapefile}.")

# Open a search cursor to find features that match the criteria
matching_features = []
with arcpy.da.SearchCursor(input_shapefile, ["SHAPE@", attribute_name]) as search_cursor:
    for row in search_cursor:
        if row[1] == attribute_value:
            matching_features.append(row[0])  # Store matching geometries

# If matching features exist, create a new shapefile to store them
if matching_features:
    spatial_reference = arcpy.Describe(input_shapefile).spatialReference  # Get spatial reference

    # Create a new feature class for filtered data
    arcpy.CreateFeatureclass_management(
        out_path=os.path.dirname(output_shapefile),
        out_name=os.path.basename(output_shapefile),
        geometry_type="POLYGON",
        spatial_reference=spatial_reference,
    )

    # Add the attribute field to the new feature class
    arcpy.AddField_management(output_shapefile, attribute_name, "TEXT")

    # Insert matching features into the new shapefile
    with arcpy.da.InsertCursor(output_shapefile, ["SHAPE@", attribute_name]) as insert_cursor:
        for geometry in matching_features:
            insert_cursor.insertRow((geometry, attribute_value))

    print(f"Filtered features saved successfully to {output_shapefile}.")
else:
    print("No matching features found.")

# Check in the 3D Analyst extension (if previously checked out)
arcpy.CheckInExtension("3D")

# Set scratch workspace as the current workspace
arcpy.env.workspace = scratch_gdb_fpath
print(f"Workspace set to scratch.gdb: {scratch_gdb_fpath}")

# Ensure ArcPy can overwrite existing files
arcpy.env.overwriteOutput = True

# Define study area shapefile path
study_area = os.path.join(data_dir_path, "study_area.shp")

# Define output file for the intersection analysis
output_intersect = os.path.join(data_dir_path, "2010_clipped_vri.shp")

# Perform an intersection between vegetation data and study area boundaries
arcpy.analysis.Intersect(["VEG_COMP_LYR_R1_POLY", study_area], output_intersect)

print(f"Intersection analysis completed. Output saved to {output_intersect}.")